In [1]:
from ipywidgets import widgets
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import json
import webbrowser
import httplib2
import re
from apiclient import discovery
from oauth2client import client
from datetime import datetime, timedelta

In [2]:
## widgets layouts
spread_item = widgets.Layout(flex='3 1 auto', width='auto')
shrink_item = widgets.Layout(flex='1 3 auto', width='auto')
col_with_50 = widgets.Layout(width='50%')
col_with_70 = widgets.Layout(width='70%')
col_with_30 = widgets.Layout(width='30%')

In [3]:
class Create_Client():
    """Creates a Google API Service Object"""

    def __init__(self):
        pass

    # Auth-Code generieren
    def get_auth_code(self, scope):
        """Runs through the OAuth flow and retrieves the code by the user"""
        self.flow = client.flow_from_clientsecrets(
            'client_secrets.json', 
            scope=scope, 
            redirect_uri='urn:ietf:wg:oauth:2.0:oob')
        auth_uri = self.flow.step1_get_authorize_url()
        webbrowser.open(auth_uri)
        print(auth_uri)

    # authorisieren
    def authorize(self, code):
        credentials = self.flow.step2_exchange(code)
        http = httplib2.Http()
        http = credentials.authorize(http)
        self.service = discovery.build('webmasters','v3',http=http)
    
    # Liste aller verifizierten Properties des Google-Accounts
    def get_verified_site_list(self):
        
        site_list = self.service.sites().list().execute()
        
        # Filter die Seiten mit falschem Permission-Level raus
        # ? was macht die zweite Bedingung: nur gültige http-Adressen?
        
        return [s['siteUrl'] for s in site_list['siteEntry'] 
                    if s['permissionLevel'] != 'siteUnverifiedUser'
                    and s['siteUrl'][:4] == 'http']
    
    # Normale GSC-Abfrage 
    # Aufruf der searchanalytics().query-Methode
    def get_gsa_data(self, _property, request):
        response = self.service.searchanalytics().query(siteUrl=_property, body=request).execute()
        if 'rows' in response:
            return self.resp2frame(response['rows'], request)
        else:
            # Wenn keine Daten vorhanden sind, dann printe einfach 'Keine Daten'
            print_log('Keine Daten', False)

    # eigene Methode: Responses to Frame
    @staticmethod
    def resp2frame(response, request):
        # Spaltennamen: Klicks, Impressionen, CTR, Position
        columns = ['clicks', 'impressions', 'ctr', 'position']
        df = pd.DataFrame(response)
        if 'keys' in df.columns:
            keys = df['keys'].apply(pd.Series)
            keys.columns = request['dimensions']
            df = df.join(keys).drop('keys', axis=1)
            columns = request['dimensions'] + columns
        df = df[columns]
        return df.sort_values('clicks', ascending=False)

In [4]:
gsa_client = Create_Client()
properties = []
today = datetime.today().date()
start_date = str(today - timedelta(90))
end_date = str(today - timedelta(2))
query = {
    "dimensions": 
    [
        "query",
        "page"
    ],
    "dimensionFilterGroups": 
    [
        {
          "filters": 
          [
            {
              "dimension": "page",
              "operator": "equals",
              "expression": ""
            }
          ]
        }
    ],
    "searchtype":"web",
    "endDate": end_date,
    "startDate": start_date,
    "rowLimit": 5000
}

In [5]:
# query['dimensionFilterGroups'][0]['filters'][0].update({'expression':'test'})

In [6]:
# Suggest: Frage die Pseudo-API von Google Suggest ab
# Nutzung von BeautifulSoup zum Auslesen des XML-Outputs
# suggestions: DatamFrame mit allen Sugesstions

def suggest(kw):
    
    sug_url = 'http://suggestqueries.google.com/complete/search?output=toolbar&hl=de&q='
    url = sug_url + str(kw)
    r = requests.get(url)
    s = bs(r.content, "lxml")
    suggestions = s.find_all('suggestion')
    suggestions = [sug['data'] for sug in suggestions]
    suggestions = pd.DataFrame(suggestions, columns=['Suggestions'])
    return suggestions  

In [7]:
# Widget Outputs

o_error = widgets.Output(wait=True)

o_log = widgets.Output(wait=True)

o_control = widgets.Output(wait=True)

o_html = widgets.Output(wait=True)

o_new = widgets.Output(wait=True)

o_gsa = widgets.Output(wait=True)

o_suggest = widgets.Output(wait=True)

In [8]:
## widgets

prop_filter = widgets.Text(placeholder='Filter', continuous_update=False)

properties_dd = widgets.Dropdown(options=properties, description='Property', layout=spread_item)

url_input = widgets.Text(description='URL', layout=spread_item, continuous_update=False)

gsc_url = widgets.Text(description='GSC URL', layout=spread_item, continuous_update=False)

startb = widgets.Button(description='Fetch URL', button_style='info')

startgsc = widgets.Button(description='GSC Abfragen', button_style='info')

keyword = widgets.Text(description='Keyword', layout=spread_item)

starts = widgets.Button(description='Suggest', button_style='info')

clears = widgets.Button(description='Clear', button_style='info')

ot_l = widgets.HTML(description='Länge', value='')

old_title_html = widgets.HTML(description='Alter Title')

od_l = widgets.HTML(description='Länge', value='')

old_desc_html = widgets.HTML(description='Alte Descr.')

new_title = widgets.Textarea(description='Neue Title', disabled=False, layout=spread_item, rows=2)

nt_l = widgets.HTML(description='Länge', value='')

new_title_html = widgets.HTML(description='Neuer Title')

new_desc = widgets.Textarea(description='Neue Descr.', disabled=False, layout=spread_item, rows=5)

nd_l = widgets.HTML(description='Länge (uncodiert)', value='')

nd_l_c = widgets.HTML(description='Länge (codiert)', value='')

nd_pixel_l = widgets.HTML(description='Pixellänge', value='')

new_desc_html = widgets.HTML(description='Neue Descr.')

## Sonderzeichen Buttons
haken = widgets.Button(description='✓', tooltip='&#10003;', layout=shrink_item, button_style='danger')

haken2 = widgets.Button(description='✔', tooltip='&#10004;', layout=shrink_item, button_style='danger')

pfeil = widgets.Button(description='➔', tooltip='&#10132;', layout=shrink_item, button_style='danger')

stern = widgets.Button(description='★', tooltip='&#9733;', layout=shrink_item, button_style='danger')

herz = widgets.Button(description='♥', tooltip='&#9829;', layout=shrink_item, button_style='danger')

telefon = widgets.Button(description='✆', tooltip='&#9990;', layout=shrink_item, button_style='danger')

flieger = widgets.Button(description='✈', tooltip='&#9992;', layout=shrink_item, button_style='danger')

zeichen = [
    haken,
    haken2,
    pfeil,
    stern,
    herz,
    telefon,
    flieger
]

In [9]:
## widgets functions
def filter_properties(change):
    options = [prop for prop in properties if change['new'] in prop]
    properties_dd.options = options
prop_filter.observe(filter_properties, names='value')

def print_log(message, clear=True):
    if clear:
        o_log.clear_output()
    with o_log:
        print(message)

def get_url_data(url):
    print_log('start')
    if url == '':
        print_log('Leere URL')
    else:
        fetch_url(url)
        print_log('finish', False)
#         old_title.value = ''
#         old_desc.value = ''
        old_desc_html.value = ''
        if metas['title'] != '':
            old_title_html.value = metas['title'].decode()
        if metas['description'] != '':
            old_desc_html.value = metas['description'].decode()
        ot_l.value = str(len(old_title_html.value))
        od_l.value = str(len(old_desc_html.value))
        nt_l.value = str(len(new_title.value))
        nd_l.value = str(len(new_desc.value))
# url_input.observe(get_url_data, names='value')

def b_start(b):
    get_url_data(url_input.value)
    
startb.on_click(b_start)


def gsc_start(b):
    get_gsa(gsc_url.value)
    o_gsa.clear_output()
    with o_gsa:
        if gsa is not None:
            
            gsa['clicks'] = gsa['clicks'].astype(int)
            gsa['impressions'] = gsa['impressions'].astype(int)
            gsa['ctr'] = gsa['ctr'].mul(100).round(2)
            gsa['position'] = gsa['position'].round(2)
        
            display(gsa[['query','clicks','impressions','ctr','position']].head(20))
            
        else: 
            print('Ungütlige URL oder keine Daten zur URL vorhanden')
             

# Klicke den Button und Starte die GSC-Abfrage
startgsc.on_click(gsc_start)

def fetch_url(url):
    try:
        print_log('fetching url', False)
        r = requests.get(url, allow_redirects=False)
        status = r.status_code
        print_log('status: ' + str(status), False)
        if status == 200:
            get_meta(r)
        elif status > 300 & status < 400:
            r2 = requests.get(url, allow_redirects=True)
            status2 = r2.status_code
            print_log('fetching url - ' + str(status) + ' ' + r2.url + ' ' + str(status2), False)
            get_meta(r2)
        else:
            get_meta()
    except Exception as e:
        print_log(e, False)

            
metas = {'title':'','description':''}
def get_meta(request = None):
    global metas
    print_log('parsing metas', False)
    title = ''
    desc = ''
    if request is not None:
        s = bs(request.content, "lxml")
        title = s.title.string.encode('utf-8')
        desc = s.find('meta', attrs={'name':'description'})['content'].encode('utf-8')
    metas.update({'title':title, 'description':desc})

gsa = pd.DataFrame()

# GSC-Daten Abfrage starten
def get_gsa(url = None):
    global gsa
    print_log('query gsa', True)
    gsa = pd.DataFrame()
    if url is None or url == '':
        print_log('Keine URL', False)
    else:
        try:
            # müssen Query trimmen, fals jemand ein vor oder nachgestelltes Leerzeichen einfügt
            # regex: (https:\/\/|http:\/\/).*\S
            m = re.search('(https:\/\/|http:\/\/).*\S', gsc_url.value)
            query['dimensionFilterGroups'][0]['filters'][0].update({'expression':m.group(0)})
            gsa = gsa_client.get_gsa_data(properties_dd.value, query)
            print_log('finish', True)
        except Exception as e:
            print_log(e, False)

def b_suggest(b):
    if keyword.value != '':
        suggests = suggest(keyword.value)
        suggests_str = widgets.HTML(value=str(' | ').join(suggests.Suggestions.tolist()))
        with o_suggest:
            display(suggests_str)
            
starts.on_click(b_suggest)

def b_clear_suggest(b):
    o_suggest.clear_output()
    
clears.on_click(b_clear_suggest)


# dynamische change-function
def new_title_length(change):
    nt_l.value=str(len(change['new']))
    new_title_html.value = str(change['new'])
    
new_title.observe(new_title_length, names='value')

# dynamische change-function
def new_desc_length(change):
    nd_l.value=str(len(change['new']))
    new_desc_html.value = str(change['new'])
    
new_desc.observe(new_desc_length, names='value')

def insert_sign(b):
    new_desc.value = new_desc.value + ' ' + b.tooltip + ' '

# Einfügen von Sonderzeichen-Symbolen
haken.on_click(insert_sign)

haken2.on_click(insert_sign)

pfeil.on_click(insert_sign)

stern.on_click(insert_sign)

herz.on_click(insert_sign)

telefon.on_click(insert_sign)

flieger.on_click(insert_sign)

In [29]:
# Fügt das Snippet-Logo ein

file = open("snippet-generator.png", "rb")
image = file.read()
widgets.Image(
    value=image,
    format='png',
    width='400'
)

A Jupyter Widget

In [41]:
# gelber Button wird erstellt
start_a = widgets.Button(description='Mit Google Konto anmelden', layout=spread_item, button_style='warning')

# Inputfeld für Authentifizierung
auth_input = widgets.Text(placeholder='Auth-Code hier einfügen', layout=spread_item)

# Button zum Autorisieren 
finish_a = widgets.Button(description='Autorisieren', layout=spread_item, button_style='success', tooltip='Jetzt authorisieren!', icon='check')

# Beim Klicken des gelben Buttons startet der Authentisierungsvorgang
def b_start_a(b):
    gsa_client.get_auth_code(['https://www.googleapis.com/auth/webmasters.readonly'])
    
# Funktionsaufruf für den Startbutton button_start_a!
start_a.on_click(b_start_a)


# Funktionshandling für den Finish-Button button_finish_a!
def b_finish_a(b):
    global properties
    try:
        gsa_client.authorize(auth_input.value)
        properties = gsa_client.get_verified_site_list()
        properties_dd.options = properties
    except Exception as e:
        o_log.clear_output()
        print_log(e, True)
        
finish_a.on_click(b_finish_a)
    
# Generiert eine H(orizontale)Box, bestehend aus start_a, auth_input, finish_a (siehe oben in Zelle)    
widgets.HBox([start_a, auth_input, finish_a], layout=widgets.Layout(justify_content='space-around'))

A Jupyter Widget

In [12]:
## control building
rows = []
rows.append(widgets.HBox([properties_dd, prop_filter]))
rows.append(widgets.HBox([gsc_url, startgsc]))
rows.append(widgets.HBox([url_input, startb]))
rows.append(widgets.HBox([keyword, starts, clears]))
left = rows
right = o_log
o_control.clear_output()
with o_control:
    display(widgets.VBox(rows))

In [13]:
## o_html building
cols = []
left = [old_title_html,ot_l,old_desc_html,od_l]
right = [new_title_html,nt_l,new_desc_html,nd_l]
cols.append(widgets.VBox(left, layout=col_with_50))
cols.append(widgets.VBox(right, layout=col_with_50))
o_html.clear_output()
with o_html:
    display(widgets.HBox(cols))

In [14]:
## o_new building
cols = []
left = [new_title,new_desc]
right_1 = zeichen[:int(len(zeichen)/2)]
right_2 = zeichen[int(len(zeichen)/2):]
right = [widgets.VBox(right_1),widgets.VBox(right_2)]
cols.append(widgets.VBox(left, layout=widgets.Layout(width='80%')))
cols.append(widgets.HBox(right, layout=widgets.Layout(width='20%')))
o_new.clear_output()
with o_new:
    display(widgets.HBox(cols))

In [15]:
## error output
o_error

A Jupyter Widget

In [16]:
## control output
o_control

A Jupyter Widget

In [17]:
o_suggest

A Jupyter Widget

In [18]:
o_html

A Jupyter Widget

In [19]:
o_new

A Jupyter Widget

In [20]:
o_gsa

A Jupyter Widget

In [21]:
## log output
o_log

A Jupyter Widget